# How to build a network?
- Social network simulation?

- We have:
    - Datetime
    - Innovation
    - Location
- How to build a Network from this?
- -> Tree shaped?

# CSV Import
- Dask only import if one of the Inno Columns is ==True; 
- Loop trough all CSV's and append "tures" to a single dd
- Turn into gpd

In [ ]:
import dask.dataframe as dd
import geopandas as gpd
import pandas as pd

: 

# Get coordinates for Tweets (Geocoding)
- Define one spatial identifier for every user (Coordinate, Place, User Location)
- Safe user and place in a separate list
- 
- Geocode each unique location

- Slight noise for tweets at same city?



# Visualize
--> Make a time based visualization for each Innovation

- Kepler.GL?